<a href="https://colab.research.google.com/github/raynardj/python4ml/blob/master/experiments/example_bugbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🐞 Customize your bug
> Bugs are, terrifying things, to be exposed to your students, especially the ones not familiar with python.

> But to fix bugs, first we have to conquer the fear. The fear over its **complexity** and **verbosity**

## A friendlier experience
Here we try to offer a smooth experience over frequently met bugs, as this can be configured freely, even outside of this library

## Import and deploy the bugbook
```python
from unpackai.bug import ishell, BUGBOOK
```
If you ran this, you'll have different experience encountering bugs:
* There will be download report button, so you can send your Mentor/TA a detailed report
* The error prompt will be more friendly structured, it will show message from [unpackai](www.unpackai.com)
* Still shows the original traceback, if needed




In [1]:
!pip install -q git+https://github.com/unpackai/unpackai

     |████████████████████████████████| 85 kB 3.9 MB/s 


## Customized message

In [2]:
from unpackai.bug import ishell, BUGBOOK, itb

For many situations, like in tabular data, if the error is a ```KeyError``` and keyword ```pandas``` appears in the traceback, it usually has only 1 meaning: we use the wrong column name that doesn't exist in the dataframe

### Before we customize the message

You can see the error message is like following, no tip from ```unpackai```

In [3]:
import pandas as pd
pd.DataFrame({"week":[1,2,3]})["weeks"]

Traceback (most recent call last):
  File "<ipython-input-3-7b7df896c955>", line 2, in <module>
    pd.DataFrame({"week":[1,2,3]})["weeks"]
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py", line 2906, in __getitem__
    indexer = self.columns.get_loc(key)
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/indexes/base.py", line 2900, in get_loc
    raise KeyError(key) from err
KeyError: 'weeks'



### Customizing KeyError

#### Simple unified message

In [4]:
# configure the bug mapping using error type name as the key (case sensitive)
BUGBOOK["KeyError"] = f"KeyError usually means there is no such key in a dictionary, but also can mean there is no such column the dataframe table"

Now check the ```UnpackAI Tips```  , congrats on your first reconfigured error report

In [5]:
import pandas as pd
pd.DataFrame({"week":[1,2,3]})["weeks"]

Traceback (most recent call last):
  File "<ipython-input-5-7b7df896c955>", line 2, in <module>
    pd.DataFrame({"week":[1,2,3]})["weeks"]
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py", line 2906, in __getitem__
    indexer = self.columns.get_loc(key)
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/indexes/base.py", line 2900, in get_loc
    raise KeyError(key) from err
KeyError: 'weeks'



#### Simple conditional message output
> Certainly we can be much more smarter than that, we can try to customize the message text according to the error detail.

> Instead of setting a **string** to ```BUGBOOK```, we can set a **function**, that will take```etype```, ```evalue```, ```tb``` as inputs, but return a string message

In [6]:
def customized_keyerror(etype, evalue, tb):
    # create structured traceback
    stb = itb.structured_traceback(etype, evalue, tb)
    # string format of trace back
    sstb = itb.stb2text(stb)

    # if this is a pandas keyerror?
    if 'pandas' in sstb:
        key = str(evalue)
        return f"""
        There's no <strong style="font-weight: bold; color: var(--ansi-red);">column {key}</strong> in the dataframe
        <br>
        <p>Try to...</p>
        """
    else:
        return f"There's no key {key} in the dictionary"

BUGBOOK["KeyError"] = customized_keyerror

Hence we got our flexibility, check the ```UnpackAI Tips``` again

In [8]:
pd.DataFrame({"week":[1,2,3]})["weeks"]

Traceback (most recent call last):
  File "<ipython-input-8-e43fbc968f48>", line 1, in <module>
    pd.DataFrame({"week":[1,2,3]})["weeks"]
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py", line 2906, in __getitem__
    indexer = self.columns.get_loc(key)
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/indexes/base.py", line 2900, in get_loc
    raise KeyError(key) from err
KeyError: 'weeks'



## About the **function** you passed in
### Inputs
Notice the input arguments of the function, it will always be
* etype: error type, ```etype.__name__=='KeyError'```
* evalue: default error message, in the above case ```str(evalue)=='weeks'```
* tb: traceback, in the above case
```python
itb.structured_traceback(etype, evalue, tb).stb2text(stb)
```
will be the entire traceback text

### Return
The return of the function will always be a string, it's actually a **HTML string** to render. So images, js, widgets, you name it...

With this flexibility, your own imagination will be the limit

## For **contributors**

### You can put bugbook under different module 🔧
If  you're developing unpackai library, eg. if you are developing nlp module

If within the NLP module, certain error can only leads to certain solution under this subject

You can put the following under ```unpackai/nlp/__init__.py```:
```python
from unpackai.bug import BUGBOOK, ishell

def customized_file_notfound(et, ev, tb):
    if 'gdrive' in str(ev):
        return f"""
        Please check if the google drive is mounted...
        <button>reload google drive</button>"""
    elif '.txt' in str(ev):
        return f"""
        Please check the text data is ...
        <button>upload text again</button>"""
    else:
        return str(ev)

BUGBOOK['FileNotFoundError'] = customized_file_notfound
```

> In such way, the error handling for CV, tabular and NLP can be **different and specific**

### You can further improve the interface 🌈
* You can change the bug page by editing [this html template](https://github.com/unpackAI/unpackai/blob/main/unpackai/static/html/bug/error_tiny_page.html)
* YOu can change the error report page by editing [this html template](https://github.com/unpackAI/unpackai/blob/main/unpackai/static/html/bug/error_report.html)


## Fancier features
> Well, if the above things didn't buffle you, let's not stop here

Now you know this format to set new configuration
```python
BUGBOOK['FileNotFoundError'] = customized_file_notfound
```

But for the key of configuration ```FileNotFoundError```, for now is a string to filter the error, means we do not apply this rule to other error, only the  ```FileNotFoundError```

This key is acting as a **filter**. The key can also be a **function** instead of a **string**

In [9]:
def pandas_filter(et, ev, tb) -> bool:
    # create structured traceback
    stb = itb.structured_traceback(et, ev, tb)
    # string format of trace back
    sstb = itb.stb2text(stb)

    # return True if the traceback has keyword pandas
    # no matter what error type it is
    if 'pandas' in sstb:
        return True
    return False

Now, use the filter function ```pandas_filter``` as the key

In [10]:
BUGBOOK[pandas_filter] = "This is a pandas error, please refer to the <a href='...'>documentation here</a>"

In [13]:
pd.DataFrame({"week":[1,2,3]}).not_a_method()

Traceback (most recent call last):
  File "<ipython-input-13-a113cb3d5dc2>", line 1, in <module>
    pd.DataFrame({"week":[1,2,3]}).not_a_method()
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py", line 5141, in __getattr__
    return object.__getattribute__(self, name)
AttributeError: 'DataFrame' object has no attribute 'not_a_method'

